In [1]:
# 建立資料庫連線
from langchain_community.utilities import SQLDatabase

user = "postgres"
password = "postgres"
host = "127.0.0.1"
port = 5432
db = "mydb"
conn_str = f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{db}"
db = SQLDatabase.from_uri(conn_str, schema="stock_market")

print(f"Dialect: {db.dialect}")
print(f"Available tables: {db.get_usable_table_names()}")
print(f'Sample output: {db.run("SELECT * FROM adj_stock_price LIMIT 5;")}')

Dialect: postgresql
Available tables: ['adj_stock_price', 'adj_stock_price_bkp', 'stock_chip_data']
Sample output: [('1101', '台泥', Decimal('20250701'), '01', Decimal('24.8134'), Decimal('25.5828'), Decimal('24.5729'), Decimal('25.1500'), Decimal('54915'), Decimal('1431632'), Decimal('197463')), ('1102', '亞泥', Decimal('20250701'), '01', Decimal('40.3043'), Decimal('41.0603'), Decimal('40.1153'), Decimal('40.9658'), Decimal('7880'), Decimal('340329'), Decimal('153744')), ('1103', '嘉泥', Decimal('20250701'), '01', Decimal('14.1381'), Decimal('14.3304'), Decimal('14.1381'), Decimal('14.1862'), Decimal('507'), Decimal('7499'), Decimal('11656')), ('1104', '環泥', Decimal('20250701'), '01', Decimal('27.8500'), Decimal('28.3000'), Decimal('27.8500'), Decimal('28.0500'), Decimal('382'), Decimal('10752'), Decimal('19261')), ('1108', '幸福', Decimal('20250701'), '01', Decimal('14.0525'), Decimal('14.2424'), Decimal('14.0525'), Decimal('14.1475'), Decimal('108'), Decimal('1624'), Decimal('6031'))]


In [2]:
# 載入模型
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [3]:
# 載入SQLDatabaseToolkit
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=model)

# 查看工具
tools = toolkit.get_tools()

for tool in tools:
    print(f"{tool.name}: {tool.description}\n")

sql_db_query: Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.

sql_db_schema: Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3

sql_db_list_tables: Input is an empty string, output is a comma-separated list of tables in the database.

sql_db_query_checker: Use this tool to double check if your query is correct before executing it. Always use this tool before executing a query with sql_db_query!



In [4]:
# 查看工具
from langchain_community.tools.sql_database.tool import (
    InfoSQLDatabaseTool,
    ListSQLDatabaseTool,
    QuerySQLCheckerTool,
    QuerySQLDatabaseTool,
)

In [5]:
from langchain_classic import hub
from langchain_core.prompts import ChatPromptTemplate
# 至LangChain Hub下載模板
# 模板連結: https://smith.langchain.com/hub/langchain-ai/sql-agent-system-prompt
template = hub.pull("langchain-ai/sql-agent-system-prompt")

# 代入參數
# dialect: 使用的SQL程式
# top_k: 返回的查詢結果資料筆數(若User未指定)
system_prompt_str = template.format(
    dialect=db.dialect,
    top_k=5
)

In [7]:
from langchain.agents import create_agent
# 建立Agent
agent = create_agent(
    model=model,
    tools=tools,
    system_prompt=system_prompt_str
)

# 測試是否成功
result = agent.invoke({"messages": [{"role": "user", "content": "hello"}]})
print(result)
print(result["messages"][-1].content)

{'messages': [HumanMessage(content='hello', additional_kwargs={}, response_metadata={}, id='ad67fc03-263f-4a8f-9c16-051505499b3b'), AIMessage(content='Hello! How can I assist you today? If you have any questions about the database or need help with SQL queries, feel free to ask!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 545, 'total_tokens': 575, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CZqeEvbHh7GJL9fS2u4fCfPyscTFx', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--d5061e16-57cc-44b1-a1e8-76fbcdfe2491-0', usage_metadata={'input_tokens': 545, 'output_tokens': 30, 'total_tokens': 575, 'input_token_d

In [8]:
# 直接回答
question = "請給我台泥股票最新的日期和收盤價"

result = agent.invoke({"messages": [{"role": "user", "content": question}]})

# 取出最後的回答
final_answer = result["messages"][-1].content
print(final_answer)

台泥股票（代碼：1101）最新的日期和收盤價如下：

- 日期：2025年10月17日，收盤價：23.35
- 日期：2025年10月16日，收盤價：23.00
- 日期：2025年10月15日，收盤價：22.95
- 日期：2025年10月14日，收盤價：22.90
- 日期：2025年10月13日，收盤價：23.00

如果需要更多資訊，請告訴我！


In [9]:
# streaming輸出
question = "請給我台泥股票最新的日期和收盤價"
# question = "請給我2330在2025/10的平均收盤價格"

from rich.console import Console
from rich.panel import Panel
from rich.syntax import Syntax
import json

console = Console()

def pretty_block(step, block):
    btype = block["type"]

    # ------- LLM TEXT -------
    if btype == "text":
        console.print(
            Panel(
                block["text"],
                title="[bold bright_green]🤖 LLM 回覆[/bold bright_green]",
                style="bright_green",
                border_style="bright_green"
            )
        )

    # ------- TOOL CALL -------
    elif btype == "tool_call":
        name = block["name"]
        args = block.get("args", {})
        args_json = json.dumps(args, ensure_ascii=False, indent=2)

        console.print(
            Panel(
                Syntax(args_json, "json", theme="monokai"),
                title=f"[bold black on bright_yellow]🔧 工具呼叫：{name}[/bold black on bright_yellow]",
                style="bright_yellow",
                border_style="bright_yellow"
            )
        )

    # ------- TOOL RESPONSE -------
    elif btype == "tool_response":
        content = block["content"]

        try:
            js = json.dumps(content, ensure_ascii=False, indent=2)
            syntax = Syntax(js, "json", theme="monokai")
        except:
            syntax = Syntax(str(content), "text", theme="monokai")

        console.print(
            Panel(
                syntax,
                title="[bold bright_cyan]📦 工具回應[/bold bright_cyan]",
                style="bright_cyan",
                border_style="bright_cyan"
            )
        )

    # ------- FALLBACK -------
    else:
        console.print(
            Panel(
                Syntax(json.dumps(block, indent=2), "json", theme="monokai"),
                title=f"[bold magenta]未知類型：{btype}[/bold magenta]",
                style="magenta",
                border_style="magenta"
            )
        )


# ============================
# main loop (不變)
# ============================
for chunk in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="updates",
):
    for step, data in chunk.items():

        # rule color
        console.rule(f"[bold bright_blue]STEP: {step}[/bold bright_blue]")

        blocks = data["messages"][-1].content_blocks
        for block in blocks:
            pretty_block(step, block)

─────────────────────────────────────────────────── STEP: model ───────────────────────────────────────────────────

╭──────────────────────────────────────── 🔧 工具呼叫：sql_db_list_tables ────────────────────────────────────────╮
│ {}                                                                                                              │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

─────────────────────────────────────────────────── STEP: tools ───────────────────────────────────────────────────

╭────────────────────────────────────────────────── 🤖 LLM 回覆 ──────────────────────────────────────────────────╮
│ adj_stock_price, adj_stock_price_bkp, stock_chip_data                                                           │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

─────────────────────────────────────────────────── STEP: model ───────────────────────────────────────────────────

...

╭─────────────────────────────────────────── 🔧 工具呼叫：sql_db_query ───────────────────────────────────────────╮
│ {                                                                                                               │
│   "query": "SELECT date, close FROM adj_stock_price WHERE code = '1101' ORDER BY date DESC LIMIT 5;"            │
│ }                                                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

─────────────────────────────────────────────────── STEP: tools ───────────────────────────────────────────────────

╭────────────────────────────────────────────────── 🤖 LLM 回覆 ──────────────────────────────────────────────────╮
│ [(Decimal('20251017'), Decimal('23.3500')), (Decimal('20251016'), Decimal('23.0000')), (Decimal('20251015'),    │
│ Decimal('22.9500')), (Decimal('20251014'), Decimal('22.9000')), (Decimal('20251013'), Decimal('23.0000'))]      │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

─────────────────────────────────────────────────── STEP: model ───────────────────────────────────────────────────

╭────────────────────────────────────────────────── 🤖 LLM 回覆 ──────────────────────────────────────────────────╮
│ 台泥股票（代碼：1101）最新的日期和收盤價如下：                                                                  │
│                                                                                                                 │
│ - 日期：2025年10月17日，收盤價：23.3500                                                                         │
│ - 日期：2025年10月16日，收盤價：23.0000                                                                         │
│ - 日期：2025年10月15日，收盤價：22.9500                                                                         │
│ - 日期：2025年10月14日，收盤價：22.9000                                                                         │
│ - 日期：2025年10月13日，收盤價：23.0000                                                                         │
│                                                                                                                 │
│ 這是最近五個交易日的數據。                                                                                      │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯